## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-21-03-09 +0000,nyt,Senator Blumenthal Opens Inquiry Into Iran Tra...,https://www.nytimes.com/2026/02/24/technology/...
1,2026-02-24-21-02-01 +0000,nypost,NYC assistant principal led double life as pim...,https://nypost.com/2026/02/24/us-news/nyc-assi...
2,2026-02-24-21-01-43 +0000,nyt,A Blizzard With Record Snowfall Slams the Nort...,https://www.nytimes.com/2026/02/24/us/record-s...
3,2026-02-24-21-00-06 +0000,nyt,‘Abolish ICE’ Wins Chicago’s Snowplow Naming C...,https://www.nytimes.com/2026/02/24/us/chicago-...
4,2026-02-24-20-59-28 +0000,nypost,‘Snowmanhenge’ and other wild blizzard sculptu...,https://nypost.com/2026/02/24/us-news/snowmanh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2528/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,trump,69
69,new,29
173,union,25
97,mexico,23
738,war,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
328,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,192
239,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...,175
379,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...,162
272,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,160
144,2026-02-24-16-59-59 +0000,nypost,Trump weighs new batch of national security ta...,https://nypost.com/2026/02/24/business/trump-w...,149


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
328,192,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...
368,90,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...
389,65,2026-02-23-22-17-00 +0000,wsj,Rep. Tony Gonzales of Texas faced calls from R...,https://www.wsj.com/politics/policy/tony-gonza...
239,61,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...
338,58,2026-02-24-01-03-34 +0000,nypost,Mexico beach named best in the world — right b...,https://nypost.com/2026/02/23/world-news/mexic...
36,58,2026-02-24-20-13-35 +0000,bbc,Mandelson's lawyers say he was arrested over '...,https://www.bbc.com/news/articles/c05vey47yddo...
11,46,2026-02-24-20-50-39 +0000,nypost,Defiant Zelensky refuses surrendering land on ...,https://nypost.com/2026/02/24/world-news/defia...
85,46,2026-02-24-19-03-59 +0000,nypost,The death of the $8 coffee! As pricey Starbuck...,https://nypost.com/2026/02/24/lifestyle/how-lu...
142,43,2026-02-24-17-00-00 +0000,nypost,OpenAI asks California watchdog to probe who’s...,https://nypost.com/2026/02/24/business/openai-...
308,40,2026-02-24-04-21-44 +0000,nyt,"Peter Mandelson, Ex-Ambassador to the U.S., Is...",https://www.nytimes.com/2026/02/23/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
